In [22]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from cv_builder.models import Resume
from dotenv import load_dotenv

load_dotenv()


llm = AzureChatOpenAI(
    deployment_name="gpt-35-16k",
    temperature=1,
    # verbose=False,
    # streaming=True,

)

# Set up a parser + inject instructions into the prompt template.
pydantic_parser = PydanticOutputParser(pydantic_object=Resume)

format_instructions = pydantic_parser.get_format_instructions()

template_string = """
You are a professional resume builder who specialized in building resumes for postgraduate applications, including master's and PhD's. 
I will provide you several piecies of information about my background and experience. You will then extract my personal information, education, work experience, research experience, project experience, and personal statement.
Below is the piece of information you will extract from:
```{texts}```

Then extract my personal information, education, work experience, research experience, project experience, and personal statement as much as you can.
{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_string)



In [23]:
texts = """
Name: John Doe | Email: john.doe@example.com | Phone: 1234567890
As a 2022 graduate, I intend to pursue postgraduate studies in New Zealand immediately after completing my undergraduate studies. Despite the impracticality of studying abroad due to the pandemic, I have taken proactive steps of enrolling in online courses during this one-year gap to enhance my knowledge in Electrical and Electronic engineering. These courses have covered essential subjects such as information technology, power systems, and green energy. Despite the challenges posed by the current circumstances, my unwavering passion for studying abroad persists. Fuelled by my determination to pursue my dreams, I am eager to apply for international study this year. My undergraduate study in Energy and Power Engineering at China Three Gorges University has instilled in me a deep fascination for electrical engineering. I aspire to pursue a master's degree in Electrical and Electronic Engineering at the University of Auckland to delve deeper into this field and become a proficient electrical engineer.

Throughout my undergraduate studies, I diligently built a solid engineering foundation by successfully completing a range of essential courses, including Engineering Fundamentals, Electrical Engineering, Engineering Mechanics, Experimental Fundamentals of Mechanical Design, Power Machinery Testing Techniques, Engineering Software Training, and New Energy Generation Technology. Additionally, I pursued a dual degree in Financial Management to merge my technical expertise with a comprehensive understanding of financial principles, equipping me with a well-rounded skillset applicable to the field of engineering.

In extracurricular activities, I actively participated in the New Media Operations Department of the College of Mechanical and Power Engineering's Youth League Committee, managing the committee's official social media account. This role cultivated valuable leadership skills and a strong work ethic. Furthermore, I achieved first prize in the university's hydrodynamic small car competition, which required effective teamwork, innovative problem-solving, and meticulous attention to detail. Engaging in these activities honed my analytical thinking, problem-solving abilities, collaborative skills, and practical application of theoretical knowledge.

During my undergraduate studies, I conducted research and completed an undergraduate thesis titled "Design of Water Turbine Guide Vane Mechanism". This endeavour demonstrated my proficiency in exploring complex topics, conducting independent research, analysing data, and delivering academic presentations. I actively participated in seminars and workshops, deepening my understanding of system design principles and techniques, high-voltage electrical systems' dynamics, and emerging intelligent distribution technologies. These experiences expanded my technical knowledge and highlighted the importance of interdisciplinary collaboration and staying updated with advancements.

In terms of my academic achievements, I have not only established a strong foundation in conducting academic research and comprehending engineering principles but have also had the privilege of working on various projects that have further ignited my passion for electrical engineering. One notable project involved collaborating with a team to design a thermal power plant for a course assignment. This experience allowed me to apply theoretical knowledge to practical engineering design and gain a deeper understanding of the complexities involved in power generation systems. Witnessing the intricate processes and challenges faced in designing a thermal power plant further fuelled my passion for pursuing research and innovation in the field. It inspired me to contribute to the development of sustainable and efficient solutions in power generation, distribution, and utilization.

The cutting-edge research conducted by the esteemed faculty in the Electrical and Electronic Engineering department at the University of Auckland captivated me. The university's research focus on power systems, renewable energy, electrical machinery, and information technology aligns perfectly with my research interests. I am enthusiastic in contributing to ongoing research projects at the forefront of advancements in Electrical and Electronic Engineering. Pursuing a master's degree in¬¬¬¬¬¬ Electrical and Electronic Engineering at the University of Auckland will equip me with the necessary skills, knowledge, and practical experience to excel in the field.

My ultimate career goal is to become a highly skilled electrical engineer who actively contributes to sustainable and efficient solutions in power generation, distribution, and utilization. With this objective in mind, I am eager to pursue the E&E master’s degree at the University of Auckland. I firmly believe that the university's research opportunities and postgraduate programs align perfectly with my academic and career goals, making it the ideal institution for me to further develop my expertise and actively contribute to the advancement of E&E Engineering. With a solid academic foundation, practical experience, and a deep-seated passion for electrical engineering, I am fully prepared for any challenge that will be involved in this programme. I am passionate about starting this exciting new phase of my academic journey, eagerly looking forward to making contribution to the vibrant academic community at the University of Auckland and spearheading innovative solutions to address the global challenges prevalent in the field of electrical systems.
"""

In [24]:
messages = prompt.format_messages(texts=texts, 
                                format_instructions=format_instructions)

In [25]:
messages

[HumanMessage(content='\nYou are a professional resume builder who specialized in building resumes for postgraduate applications, including master\'s and PhD\'s. \nI will provide you several piecies of information about my background and experience. You will then extract my personal information, education, work experience, research experience, project experience, and personal statement.\nBelow is the piece of information you will extract from:\n```\nName: John Doe | Email: john.doe@example.com | Phone: 1234567890\nAs a 2022 graduate, I intend to pursue postgraduate studies in New Zealand immediately after completing my undergraduate studies. Despite the impracticality of studying abroad due to the pandemic, I have taken proactive steps of enrolling in online courses during this one-year gap to enhance my knowledge in Electrical and Electronic engineering. These courses have covered essential subjects such as information technology, power systems, and green energy. Despite the challenge

In [26]:
output = llm(messages)

In [27]:
output.content

'{\n  "personal_info": {\n    "name": "John Doe",\n    "email": "john.doe@example.com",\n    "phone": "1234567890"\n  },\n  "education": [\n    {\n      "degree": "Undergraduate",\n      "university": "China Three Gorges University",\n      "graduation_date": "2022"\n    }\n  ],\n  "work_experience": [\n    {\n      "company": "College of Mechanical and Power Engineering\'s Youth League Committee",\n      "position": "New Media Operations Department",\n      "start_date": "Unknown",\n      "end_date": "Unknown",\n      "summary": "Managed the committee\'s official social media account",\n      "bullet_points": []\n    }\n  ],\n  "research_experience": [\n    {\n      "organization": "Unknown",\n      "position": "Undergraduate researcher",\n      "start_date": "Unknown",\n      "end_date": "Unknown",\n      "summary": "Conducted research and completed an undergraduate thesis on the topic \'Design of Water Turbine Guide Vane Mechanism\'",\n      "bullet_points": []\n    }\n  ],\n  "proj

In [28]:
resume = pydantic_parser.parse(output.content)

resume

Resume(personal_info=PersonalInfo(name='John Doe', email='john.doe@example.com', phone='1234567890'), education=[Education(degree='Undergraduate', university='China Three Gorges University', graduation_date='2022')], work_experience=[WorkExperience(company="College of Mechanical and Power Engineering's Youth League Committee", position='New Media Operations Department', start_date='Unknown', end_date='Unknown', summary="Managed the committee's official social media account", bullet_points=[])], research_experience=[ResearchExperience(organization='Unknown', position='Undergraduate researcher', start_date='Unknown', end_date='Unknown', summary="Conducted research and completed an undergraduate thesis on the topic 'Design of Water Turbine Guide Vane Mechanism'", bullet_points=[])], project_experience=[ProjectExperience(name='Thermal Power Plant Design', start_date='Unknown', end_date='Unknown', summary='Collaborated with a team to design a thermal power plant for a course assignment', bu

In [29]:
import json
with open("input.json", "w") as f:
    json.dump(resume.dict(), f)